<a href="https://colab.research.google.com/github/nikhil-xyz/Natural_Language_Processing/blob/main/Comment_Toxicity_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/nikhil-xyz/Natural_Language_Processing.git

fatal: destination path 'Natural_Language_Processing' already exists and is not an empty directory.


In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [7]:
# device = tf.test.gpu_device_name()
# device

In [8]:
df = pd.read_csv('/content/Natural_Language_Processing/data/Comment_Toxicity/train.csv')

In [9]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
df.iloc[3]['comment_text']

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

In [11]:
df[df.columns[2:]].iloc[3]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 3, dtype: int64

In [12]:
df[df['insult']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0
55,0020e7119b96eeeb,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0
56,0020fd96ed3b8c8b,=Tony Sidaway is obviously a fistfuckee. He lo...,1,0,1,0,1,0


In [13]:
from tensorflow.keras.layers import TextVectorization

In [14]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [15]:
max_features = 200000

In [16]:
vectorizer = TextVectorization(max_tokens = max_features, output_sequence_length=1800, output_mode='int')

In [17]:
vectorizer.adapt(X.values)

In [18]:
vectorized_text = vectorizer(X.values)

In [19]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [20]:

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [21]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [22]:
train = dataset.take(int(len(dataset)*0.7))
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2))
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.2))

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Embedding

In [24]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(125, activation='relu'))

model.add(Dense(6, activation='sigmoid'))

In [25]:
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
history = model.fit(train, epochs=10, validation_data=val)

Epoch 1/10
6981/6981 [==============================] - 663s 95ms/step - loss: 0.0591 - accuracy: 0.9861 - val_loss: 0.0475 - val_accuracy: 0.9950
Epoch 2/10
6981/6981 [==============================] - 666s 95ms/step - loss: 0.0457 - accuracy: 0.9911 - val_loss: 0.0412 - val_accuracy: 0.9940
Epoch 3/10
6981/6981 [==============================] - 658s 94ms/step - loss: 0.0408 - accuracy: 0.9900 - val_loss: 0.0360 - val_accuracy: 0.9943
Epoch 4/10
6981/6981 [==============================] - 654s 94ms/step - loss: 0.0367 - accuracy: 0.9939 - val_loss: 0.0312 - val_accuracy: 0.9940
Epoch 5/10
6981/6981 [==============================] - 627s 90ms/step - loss: 0.0327 - accuracy: 0.9939 - val_loss: 0.0299 - val_accuracy: 0.9948
Epoch 6/10
6981/6981 [==============================] - 612s 88ms/step - loss: 0.0296 - accuracy: 0.9907 - val_loss: 0.0254 - val_accuracy: 0.9898
Epoch 7/10
6981/6981 [==============================] - 661s 95ms/step - loss: 0.0262 - accuracy: 0.9918 - val_loss: 0

In [33]:

history = (model.predict(test) > 0.5).astype(int)

998/998 [==============================] - 49s 48ms/step


In [36]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [37]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [38]:
for batch in test.as_numpy_iterator():
  X_true, y_true = batch
  yhat = (model.predict(X_true) > 0.5).astype(int)

  y_true = y_true.flatten()
  yhat = yhat.flatten()

  pre.update_state(y_true, yhat)
  re.update_state(y_true, yhat)
  acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 55ms/step


In [39]:
print(pre.result().numpy())
print(re.result().numpy())
print(acc.result().numpy())

0.92700934
0.8979367
0.91282564


In [40]:
model.save('models/Comment_Toxicity.h5')

In [41]:
!pip install gradio jinja2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [46]:
def score_comment(comment):
  vectorized_comment = vectorizer([comment])
  result = model.predict(vectorized_comment)

  text = ''
  for index, col in enumerate(df.columns[2:]):
    text = '{}: {}\n'.format(col, result[0][index] > 0.5)
  return text

In [47]:
import gradio as gr

In [48]:
interface = gr.Interface(fn=score_comment, 
                inputs=gr.inputs.Textbox(lines=2, placeholder='comment somenthing'),
                outputs='text')
interface.launch(share=True)

/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c42b93d3-b67a-4a0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
